In [20]:
batch_size_for_classification = 8000
#prediction
prediction_path = "Export/cdnow/cdnow_prediction_theta_variantA_"+str(batch_size_for_classification+2000)+'_.csv'

In [21]:
import catboost
import pandas as pd
import numpy as np
import datetime
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import rpy2
import math
import time
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_squared_error
import time
import pickle
import warnings
warnings.filterwarnings("ignore")

# CDNOW

In [22]:
# Path fo import files
#input data
import_data = "import/CDNOW_master.txt"
# clusters
import_clusters = "clusters/results_cdnow.csv"

### 0. Preprocessing cdnow

In [23]:
df = pd.read_csv(import_data,header=None,names  =['id','Date','disks','price_purchase'],delim_whitespace=True,)
df['Date'] = df["Date"].apply(lambda x:datetime.datetime.strptime(str(x),"%Y%m%d"))
df = df.drop(columns = ['disks'])
df = df.groupby(by=['id','Date']).sum().reset_index()
df = df.pivot(columns = 'Date',index = 'id',values = 'price_purchase')
df.sample(2)

Date,1997-01-01 00:00:00,1997-01-02 00:00:00,1997-01-03 00:00:00,1997-01-04 00:00:00,1997-01-05 00:00:00,1997-01-06 00:00:00,1997-01-07 00:00:00,1997-01-08 00:00:00,1997-01-09 00:00:00,1997-01-10 00:00:00,...,1998-06-21 00:00:00,1998-06-22 00:00:00,1998-06-23 00:00:00,1998-06-24 00:00:00,1998-06-25 00:00:00,1998-06-26 00:00:00,1998-06-27 00:00:00,1998-06-28 00:00:00,1998-06-29 00:00:00,1998-06-30 00:00:00
id,,,,,,,,,,,,,,,,,,,,,
7872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4963,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df.shape

(23570, 546)

## 0.1 Choose TS with given clusters and divide into test/train dataset

In [5]:
cluster_TS  = pd.read_csv(import_clusters)
cluster_TS.head()

,Unnamed: 0,clusters_r,clusters_m,clusters_f
0,11770,0,0,0
1,14675,17,0,8
2,10026,2,0,3
3,13997,1,0,2
4,741,0,0,0


In [6]:
df_train = cluster_TS.merge(df,left_on = "Unnamed: 0", right_on = 'id').drop(columns = ['clusters_r','clusters_m','clusters_f'])
df_train = df_train.rename(index = str, columns={"Unnamed: 0": "id"}).set_index('id')
df_train.head()

,1997-01-01 00:00:00,1997-01-02 00:00:00,1997-01-03 00:00:00,1997-01-04 00:00:00,1997-01-05 00:00:00,1997-01-06 00:00:00,1997-01-07 00:00:00,1997-01-08 00:00:00,1997-01-09 00:00:00,1997-01-10 00:00:00,...,1998-06-21 00:00:00,1998-06-22 00:00:00,1998-06-23 00:00:00,1998-06-24 00:00:00,1998-06-25 00:00:00,1998-06-26 00:00:00,1998-06-27 00:00:00,1998-06-28 00:00:00,1998-06-29 00:00:00,1998-06-30 00:00:00
id,,,,,,,,,,,,,,,,,,,,,
11770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14675,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
741,NaN,NaN,NaN,43.13,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_test= df.drop(df_train.index.values,axis=0).sample(batch_size_for_classification)
df_test.head()

Date,1997-01-01 00:00:00,1997-01-02 00:00:00,1997-01-03 00:00:00,1997-01-04 00:00:00,1997-01-05 00:00:00,1997-01-06 00:00:00,1997-01-07 00:00:00,1997-01-08 00:00:00,1997-01-09 00:00:00,1997-01-10 00:00:00,...,1998-06-21 00:00:00,1998-06-22 00:00:00,1998-06-23 00:00:00,1998-06-24 00:00:00,1998-06-25 00:00:00,1998-06-26 00:00:00,1998-06-27 00:00:00,1998-06-28 00:00:00,1998-06-29 00:00:00,1998-06-30 00:00:00
id,,,,,,,,,,,,,,,,,,,,,
12838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18446,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### NAN -> 0

In [8]:
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)
df_train.sample(2)

,1997-01-01 00:00:00,1997-01-02 00:00:00,1997-01-03 00:00:00,1997-01-04 00:00:00,1997-01-05 00:00:00,1997-01-06 00:00:00,1997-01-07 00:00:00,1997-01-08 00:00:00,1997-01-09 00:00:00,1997-01-10 00:00:00,...,1998-06-21 00:00:00,1998-06-22 00:00:00,1998-06-23 00:00:00,1998-06-24 00:00:00,1998-06-25 00:00:00,1998-06-26 00:00:00,1998-06-27 00:00:00,1998-06-28 00:00:00,1998-06-29 00:00:00,1998-06-30 00:00:00
id,,,,,,,,,,,,,,,,,,,,,
13447,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Split into train/test (last 70 days) and scaling

In [9]:
train_targets, test_targets= train_test_split((df_test.append(df_train)).transpose(), test_size=.128, shuffle=False)

In [10]:
test_targets.shape

(70, 10000)

In [11]:
test_targets = test_targets.transpose()
train_targets = train_targets.transpose()


In [12]:
#scaling
scl = MinMaxScaler()
train_targets_scaled = scl.fit_transform(train_targets)

## Theta method

In [13]:
from rpy2.robjects.packages import importr
# import R's "base" package
# import rpy2's package module
import rpy2.robjects.packages as rpackages
import rpy2.robjects as robjects
# import R's utility package
utils = rpackages.importr('utils')

# select a mirror for R packages
utils.chooseCRANmirror(ind=1) # select the first mirror in the list
# R package names
packnames = ['forecast']
# R vector of strings
from rpy2.robjects.vectors import StrVector
try:
    base = importr('forecast')
except:
    utils.install_packages(StrVector('forecast'))
    base = importr('forecast')


### We take test as last 10 weeks (70 days)

In [14]:
%%time
df_prediction = pd.DataFrame(columns = test_targets.columns)
train_targets_scaled = pd.DataFrame(train_targets_scaled)
train_targets_scaled.index = test_targets.index
for index, row in train_targets_scaled.iterrows():
    res = robjects.FloatVector(row.drop(columns = ['uid']))
    r_prediction = base.thetaf(res,70)[1]
    df_prediction.loc[index] = r_prediction

CPU times: user 2min 20s, sys: 136 ms, total: 2min 21s
Wall time: 3min 2s


In [15]:
# Score is better without scl.transform(df_prediction)
print("RMSE score:", round(math.sqrt(mean_squared_error(test_targets,df_prediction)),2))

RMSE score: 2.78


## Transform prediction for output

In [16]:
df_prediction.columns  = test_targets.columns 
df_prediction.index = test_targets.index
#MODEL
df_prediction.insert(0,'RMSE',math.sqrt(mean_squared_error(test_targets,df_prediction)))
df_prediction.insert(0,'model','theta')
#METHOD
df_prediction.insert(0,'method','all_data')
#STEP
df_prediction.insert(0,'step','all_data')
# CLUSTER
df_prediction.insert(0,'cluster_id','none')
#Wide to logn
df_prediction = df_prediction.reset_index().melt(id_vars =['id','cluster_id','step','method','model','RMSE']).rename(index = str, columns={'variable':'date','value':'predicted_value'})
true_values=test_targets.reset_index().melt(id_vars=['id']).rename(index = str, columns={'variable':'date','value':'true_value'})
df_prediction = df_prediction.merge(true_values,left_on = ['id','date'],right_on=['id','date'])

### Save to file

In [17]:
df_prediction.to_csv(prediction_path)
df_prediction.sample(5)

,id,cluster_id,step,method,model,RMSE,date,predicted_value,true_value
526058,1915,none,all_data,all_data,theta,2.782111,1998-06-13,-0.000518,0.0
245904,311,none,all_data,all_data,theta,2.782111,1998-05-16,-0.001507,0.0
309289,13210,none,all_data,all_data,theta,2.782111,1998-05-22,0.000648,0.0
172702,14953,none,all_data,all_data,theta,2.782111,1998-05-09,-0.000280,0.0
411509,21355,none,all_data,all_data,theta,2.782111,1998-06-02,-0.000141,0.0


In [18]:
df_prediction.shape

(700000, 9)

In [19]:
# To get full analysis, please run "combine_predictions.ipynb" file, which combines the prediction files of all approaches